In [134]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt

## Análisis Descriptivo y Versionado Inicial de Datos

### Verificación de dimensiones y valores faltantes

In [32]:
df_unificado = pd.read_csv("data/dataset_unificado.csv")

In [33]:
df_unificado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1212 entries, 0 to 1211
Columns: 412 entries, dia to HORA
dtypes: float64(410), object(2)
memory usage: 3.8+ MB


In [36]:
df_unificado['Frio (Kw)__Consolidado EE'].describe()

count    1.212000e+03
mean     2.822680e+04
std      1.595821e+05
min      2.310000e+03
25%      1.801125e+04
50%      2.369500e+04
75%      2.926600e+04
max      5.572801e+06
Name: Frio (Kw)__Consolidado EE, dtype: float64

In [29]:
df_unificado.describe()

,Meta EE CO2__Consolidado KPI,Meta ET Bodega__Consolidado KPI,Meta ET Linea 3__Consolidado KPI,ET Cocina/Hl__Consolidado KPI,Aire Cocina / Hl__Consolidado KPI,Meta Agua Bodega__Consolidado KPI,Agua Bodega / Hl__Consolidado KPI,Meta Agua Linea 5__Consolidado KPI,Aire Envasado / Hl__Consolidado KPI,Meta Agua Elab__Consolidado KPI,...,Tot A130/330/430__Totalizadores Glicol,Tot Reposo Superior__Totalizadores Glicol,Tot Trasiego__Totalizadores Glicol,Tot A40/240/50/60/Centec/Filtro__Totalizadores Glicol,Tot A10/20__Totalizadores Glicol,Tot L3. L4 y Planta de CO2__Totalizadores Glicol,Tot Reposo Inferior__Totalizadores Glicol,"Tot L3, L4 y Planta de CO2__Totalizadores Glicol",Tot Fermantacion_Cocina__Totalizadores Glicol,Frio
count,846.000000,846.000000,846.000000,1211.000000,1211.000000,846.000000,1211.000000,846.000000,1211.000000,846.000000,...,846.000000,365.000000,1211.000000,846.000000,846.000000,365.000000,365.000000,481.000000,365.000000,1.211000e+03
mean,0.252890,0.977485,14.662274,8.747851,1.270441,0.788699,1.197842,1.478810,1.964348,1.804148,...,4843.361689,135.924452,1466.402379,2917.337711,2108.569707,1203.470378,5674.086671,3619.402298,1385.605932,2.824205e+04
std,0.027177,0.125520,1.882807,10.693978,2.538970,0.052117,2.881043,0.097719,2.983709,0.119217,...,2083.511012,561.465994,8518.488527,2205.598058,2608.291534,1001.517477,2002.975481,4978.676091,1452.177896,1.596471e+05
min,0.217857,0.863636,12.954545,0.000000,0.000000,0.685714,0.294227,1.285714,0.020958,1.568571,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,170.630000,3.750000,224.910000,2.310000e+03
25%,0.228571,0.893939,13.409091,5.136334,0.733405,0.754286,0.706485,1.414286,1.237705,1.725429,...,4201.617500,1.400000,589.892000,1262.767500,0.000000,606.090000,4410.960000,469.730000,604.700000,1.802400e+04
50%,0.242857,0.924242,13.863636,8.659836,0.875747,0.777143,0.874375,1.457143,1.595545,1.777714,...,5067.510000,3.550000,1161.410000,2098.125000,0.000000,1017.660000,5633.530000,1161.980000,963.380000,2.370100e+04
75%,0.277679,1.022917,15.343750,10.709533,1.117030,0.820952,1.165255,1.539286,2.060899,1.877929,...,5974.593750,12.610000,1694.125000,3940.692500,4880.862500,1443.190000,6959.130000,5116.750000,1515.620000,2.927500e+04
max,0.310357,1.272727,19.090909,232.038835,56.651068,0.953143,93.785714,1.787143,87.277857,2.180314,...,11039.500000,4054.270000,296218.410000,10498.000000,8110.880000,9275.440000,11066.500000,27378.380000,10870.690000,5.572801e+06


In [101]:
df_unificado.isna().sum().sort_values(ascending=False).head(119)

Unnamed: 35__Consolidado KPI                             1212
Unnamed: 103__Consolidado KPI                            1212
 __Consolidado KPI                                       1212
Unnamed: 22__Consolidado KPI                             1212
EE Linea 5 / Hl__Consolidado KPI                         1212
                                                         ... 
Meta Agua Envas__Consolidado KPI                          365
Nivel Silo Bagazo Norte (1)__Totalizadores Produccion     309
KW Trafo 8__Totalizadores Energia                          13
Totalizador Bba P1__Totalizadores Efluentes                 5
Totalizador Bba P51__Totalizadores Efluentes                5
Length: 119, dtype: int64

In [103]:
na_counts = df_unificado.isna().sum()
vars_300 = na_counts[na_counts >= 300].index.tolist()
df_unificado = df_unificado.drop(columns=vars_300, errors="ignore")

In [105]:
# Cuenta ceros
zero_counts = (df_unificado.eq(0) | df_unificado.eq("0")).sum(axis=0)
zero_ratio  = ((zero_counts / len(df_unificado)) * 100).sort_values(ascending=False)

umbral = 50.0
muchos_ceros = zero_ratio[zero_ratio >= umbral]

print("\nColumnas con ≥", umbral, "% ceros:", len(muchos_ceros), 'de', len(df_unificado.columns), "columnas totales")
print()
print(muchos_ceros.apply(lambda x: f"{x:.2f}%"))


Columnas con ≥ 50.0 % ceros: 53 de 296 columnas totales

VAPOR DE CALDERA 1 KG__Totalizadores Gas y Vapor           100.00%
Fuel Oil Tk2 (Kg)__Totalizadores Gas y Vapor               100.00%
Tot_Vapor_Caldera 3__Totalizadores Gas y Vapor             100.00%
Fuel Oil Tk1 (Kg)__Totalizadores Gas y Vapor               100.00%
HL Mosto Malta__Totalizadores Produccion                   100.00%
HL Mosto Fuerte__Totalizadores Produccion                  100.00%
HL Mosto Budweiser__Totalizadores Produccion               100.00%
HL Mosto Palermo__Totalizadores Produccion                 100.00%
HL Mosto Frost__Totalizadores Produccion                   100.00%
HL Mosto Indio__Totalizadores Produccion                   100.00%
HL Mosto Local__Totalizadores Produccion                   100.00%
Red Barriles__Totalizadores Agua                           100.00%
HL Mosto Bieckert__Totalizadores Produccion                100.00%
Agua Filt FMaCist CE__Totalizadores Agua                   100.00%
Kw l

In [106]:
cols_muchos_ceros = muchos_ceros.index.tolist()
df_unificado = df_unificado.drop(columns=cols_muchos_ceros, errors="ignore")

### Detección de valores atípicos

Totales detectados por cada método:
    columna  outliers_IQR  outliers_Zmod
0   TOTAL         10846          15428

In [128]:
# --- 1) Máscara de outliers por IQR, columna a columna ---
def iqr_mask(col, k=1.5):
    s = col.dropna()
    if s.empty:
        return pd.Series(False, index=col.index)
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    if iqr == 0:
        return pd.Series(False, index=col.index)
    low, high = q1 - k*iqr, q3 + k*iqr
    return (col < low) | (col > high)

num_cols = df_unificado.select_dtypes(include=[np.number]).columns

mask_df = pd.DataFrame(False, index=df_unificado.index, columns=df_unificado.columns)
for c in num_cols:
    mask_df[c] = iqr_mask(df_unificado[c])

# --- 2) Tabla con las celdas outliers: fila, columna, valor ---
pos = mask_df[num_cols].stack()
pos = pos[pos]  # solo True
outliers_tabla = pd.DataFrame({
    "fila":   [i for i, _ in pos.index],
    "columna":[c for _, c in pos.index],
    "valor":  [df_unificado.loc[i, c] for i, c in pos.index],
})

# --- 3) Crear df con outliers -> NaN ---
df_nan = df_unificado.copy()
df_nan[num_cols] = df_nan[num_cols].mask(mask_df[num_cols])

# --- 4) Imputación ---
imp = KNNImputer(n_neighbors=5, weights="distance")

num = df_nan.select_dtypes(include='number')              # solo numéricas
imp_vals = imp.fit_transform(num)                         # → numpy array

df_imp_num = pd.DataFrame(imp_vals, index=num.index, columns=num.columns)

# reconstruyo el dataframe completo (no numéricas se mantienen)
df_imputado = df_nan.copy()
df_imputado[num.columns] = df_imp_num

# (Opcional) ver cuántos NaN imputaste
print("\nCantidad de valores imputados:", int(mask_df[num_cols].sum().sum()))


Cantidad de valores imputados: 10846


In [131]:
df_imputado['Frio'] = df_imputado['Frio (Kw)__Consolidado EE'].shift(-1)
df_imputado = df_imputado.iloc[1:-1].reset_index(drop=True)

In [132]:
from tools import checksum
checksum(df_imputado, 'df_imputado')

'Checksum calculado y guardado: 64eed142c21e11ba2a0422f012483bcb'

##  Análisis Temporal